<a target="_blank" href="https://colab.research.google.com/github/jmanuelc87/nmp-autoavanza/blob/main/notebooks/MontePiedad_Rules.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Imports and Definitions

In [1]:
import os
import dotenv
import base64
import datetime
import itertools
import functools

from pydantic import BaseModel, Field

from operator import itemgetter

from rapidfuzz import fuzz, utils

from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel, RunnablePick
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.image import UnstructuredImageLoader

In [2]:
env = dotenv.find_dotenv()
enable_open_ai = True
enable_lang_smith = False

In [3]:
if enable_open_ai:
    os.environ["OPENAI_ORG_ID"] = dotenv.get_key(env, "org_id")
    os.environ["OPENAI_PROJECT_ID"] = dotenv.get_key(env, "proj_id")
    os.environ["OPENAI_API_KEY"] = dotenv.get_key(env, "api_key")
else:
    os.environ["OPENAI_ORG_ID"] = "***"
    os.environ["OPENAI_PROJECT_ID"] = "***"
    os.environ["OPENAI_API_KEY"] = "***"

if enable_lang_smith:
    os.environ["LANGSMITH_TRACING"] = "true"
    os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGSMITH_API_KEY"] = dotenv.get_key(env, "langsmith_api_key")
    os.environ["LANGSMITH_PROJECT"] = "pr-husky-stencil-39"

In [4]:
llm = ChatOpenAI(
    # model="*",
    model = "gpt-4.1-mini",
    temperature=0.17,
    # base_url='http://localhost:1234/v1',
)

In [5]:
extract = [
    {
        "role": "system",
        "content": "Eres un asistente servicial, usando OCR extraes los attributos que se te indiquen de la imagen y verificas que sean correctos comparandolo con el documento {doc_name}.",
    },
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": {"url": "data:image/jpg;base64,{image}"}},
            {"type": "text", "text": "El documento {doc_name}: {page_content}"},
        ],
    },
]

In [6]:
def load_image(inputs):
    """Load image from file and encode it as base64."""
    image_path = inputs["filename"]
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    image_base64 = encode_image(image_path)
    return image_base64

In [7]:
def load_unstructured_image(args):
    loader = UnstructuredImageLoader(args["filename"])
    data = loader.load()
    return data.pop().page_content

In [8]:
loader1 = RunnableLambda(load_unstructured_image)
loader2 = RunnableLambda(load_image)

### Extraccion Factura

In [9]:
image1_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 2_TK 63075-1 FAC_FRENTE_otsu.jpg"

In [10]:
prompt_extraction = ChatPromptTemplate(messages=extract)

In [11]:
class Factura(BaseModel):
    nombre_cliente: str = Field(description="El nombre del cliente")
    marca: str = Field(description="La marca")
    linea: str = Field(description="La linea")
    modelo: int = Field(description="El modelo")
    clase: str = Field(description="La clase")
    tipo: str = Field(description="El tipo")
    combustible: str = Field(description="El tipo de combustible")
    no_serie: str = Field(description="El numero de serie, NIV o VIN")
    no_motor: str = Field(description="El numero de motor")

In [12]:
llm_with_fac_structured_output = llm.with_structured_output(Factura)

In [13]:
extract_fac_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter("doc_name"),
    }
    | prompt_extraction
    | llm_with_fac_structured_output
)

### Extraccion Tarjeta de Circulacion

In [14]:
image2_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 2_TK 63075-5 TC_FRENTE_crop_binary.jpg"

In [15]:
class TarjetaCirculacion(BaseModel):
    propietario: str = Field(description="El propietario o nombre del cliente")
    vehiculo: str = Field(description="El tipo de vehiculo")
    marca: str = Field(description="La marca")
    modelo: int = Field(description="El modelo del vehiculo")
    no_motor: str = Field(description="El numero de serie del motor")
    no_niv: str = Field(description="El numero de identificacion vehicular")
    expedicion: str = Field(description="La fecha de expedicion")
    vigencia: str = Field(description="La fecha de vigencia")
    placa: str = Field(description="El numero de placa")

In [16]:
llm_with_tc_structured_output = llm.with_structured_output(TarjetaCirculacion)

In [17]:
extract_tc_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_tc_structured_output
)

### Extraccion INE

In [18]:
image3_path = "./data/bronze/BASE_AUTOAVANZA/documentos_clean/INE_FRENTE/Caso 2_TK 63075-3 INE_FRENTE_crop_binary.jpg"

In [19]:
class CredencialVotar(BaseModel):
    nombre: str = Field(description="El nombre de la persona")
    domicilio: str = Field(description="El domicilio de la persona")
    emision: int = Field(description="La fecha de emision")
    vigencia: int = Field(description="La fecha de vigencia")

In [20]:
llm_with_ine_structured_output = llm.with_structured_output(CredencialVotar)

In [21]:
extract_ine_chain = (
    {
        "page_content": loader1,
        "image": loader2,
        "doc_name": itemgetter('doc_name'),
    }
    | prompt_extraction
    | llm_with_ine_structured_output
)

# Reglas para validar documentos

Se definen ciertas reglas para validar documentos como se especifica a continuacion.

1. Se debe verificar la coincidencia del nombre del solicitante en la INE, Tarjeta de circulacion y Factura.

In [22]:
def compare_doc_names(factura: Factura, tarjeta_circulacion: TarjetaCirculacion, credencial_votar: CredencialVotar, **kwargs) -> str:
    """Compara el propietario o nombre en los documentos Factura, Tarjeta de Circulacion y Credencial para Votar

    Args:
        factura (Factura): La factura del vehiculo
        tarjeta_circulacion (TarjetaCirculacion): La Tarjeta de circulacion
        credencial_votar (CredencialVotar): La Credencial para Votar

    Returns:
        str: El resultado de la validacion es aprovado o rechazado 
    """
    wr1 = fuzz.WRatio(factura.nombre_cliente, tarjeta_circulacion.propietario, processor=utils.default_process)
    wr2 = fuzz.WRatio(factura.nombre_cliente, credencial_votar.nombre, processor=utils.default_process)
    wr3 = fuzz.WRatio(tarjeta_circulacion.propietario, credencial_votar.nombre, processor=utils.default_process)

    if wr1 >= 90. and wr2 >= 90. and wr3 >= 90.:
        return "El nombre del propietario es correcto!"
    elif 70. <= wr1 < 90. and 70. <= wr2 < 90. and 70. <= wr3 < 90.:
        return "Favor de verificar que el nombre aparezca correctamente en los tres documentos"
    else:
        return "El nombre no aparece correctamente en los documentos"

2. Se debe verificar el NIV es el mismo en la tarjeta de circulación y factura

In [23]:
def compare_vin(factura: Factura, tarjeta_circulacion:TarjetaCirculacion, **kwargs) -> str:
    """Compara el numero de serie, NIV o VIN en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """

    wr = fuzz.WRatio(factura.no_serie, tarjeta_circulacion.no_niv)

    result = []

    if wr >= 90.:
        result.append("El NIV del vehiculo es correcto!")
    else:
        result.append("El NIV es distinto. Se tiene que rechazar el tramite!")

    wr2 = fuzz.WRatio(factura.no_motor, tarjeta_circulacion.no_motor)

    if wr >= 90.:
        result.append("El numero de serie del motor es correcto!")
    else:
        result.append("El numero de serie del motor no coincide")

    return result

3. Se debe comparar los datos del vehiculo (marca, linea y modelo) en la tarjeta de circulacion y factura

In [24]:
def compare_vehicle_data(factura: Factura, tarjeta_circulacion: TarjetaCirculacion, **kwargs) -> str:
    """Compara la marca, linea y modelo del vehiculo en la factura y tarjeta de circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es aprovado o rechazado
    """
    result = []

    wr = fuzz.WRatio(factura.marca, tarjeta_circulacion.marca)

    if wr < 90.0:
        label = f"La marca del vehiculo es distinto. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(factura.linea, tarjeta_circulacion.vehiculo)

    if wr < 90.0:
        label = "La linea del vehiculo es distinta. Favor de verificar"
        result.append(label)

    wr = fuzz.WRatio(str(factura.modelo), str(tarjeta_circulacion.modelo))

    if wr < 90.0:
        label = "El modelo del vehiculo es distinto. Favor de verificar"
        result.append(label)

    if len(result) == 0:
        result.append("Los datos del vehiculo son correctos!")

    return result

9. Se debe verificar el numero de motor contra la tarjeta de circulacion

In [25]:
def compare_motor(factura: Factura, tarjeta_circulacion: TarjetaCirculacion, **kwargs) -> str:
    """Compara el motor en la Factura y Tarjeta de Circulacion

    Args:
        factura (Factura): La Factura
        tc (TarjetaCirculacion): La Tarjeta de Circulacion

    Returns:
        str: El resultado de la validacion es approvado o rechazado
    """
    wr = fuzz.WRatio(factura.no_motor, tarjeta_circulacion.no_motor)
    
    if wr < 90.0:
        return "El numero de motor del vehiculo es distinto. Favor de verificar"
    
    return "El numero de motor es correcto!"

In [26]:
def check_validity(credencial_votar: CredencialVotar, **kwargs) -> str:
    """Verfica la validez de la credencial para votar

    Args:
        ine (CredencialVotar): La credencial para votar

    Returns:
        str: El resultado de la validacion es approvador o rechazado
    """
    current_year = datetime.datetime.now().year
    if current_year > credencial_votar.vigencia:
        return "La Credencial para Votar no esta vigente"
    return "La Credencial para Votar es vigente"

In [27]:
# All the functions need to receive the `kwargs` arguments for it to work in the validate_documents loop.
tools=[compare_doc_names, compare_vin, compare_vehicle_data, compare_motor, check_validity]

In [28]:
def validate_documents(inputs, tools):
    """Validacion de los documentos

    Args:
        inputs (dict): Los datos proporcionados por el llm
        tools (list): Las herramientas que validaran los documentos

    Yields:
        str: el resultado de la validacion 
    """
    results = []
    for tool in tools:
        result = tool(**inputs)
        if isinstance(result, list):
            results.extend(result)
        else:
            results.append(result)
    return results

In [29]:
validation = RunnableLambda(functools.partial(validate_documents, tools=tools))

In [30]:
chain = {
    "factura": {
        "filename": itemgetter("factura"),
        "doc_name": lambda x: "Factura",
    }
    | extract_fac_chain,
    "tarjeta_circulacion": {
        "filename": itemgetter("tc"),
        "doc_name": lambda x: "Tarjeta de Circulacion",
    }
    | extract_tc_chain,
    "credencial_votar": {
        "filename": itemgetter("ine"),
        "doc_name": lambda x: "Credencial para Votar"
    }
    | extract_ine_chain,
} | validation

## Caso 2

In [31]:
chain.invoke(
    input={
        "factura": image1_path,
        "tc": image2_path,
        "ine": image3_path,
    }
)

['El nombre del propietario es correcto!',
 'El NIV del vehiculo es correcto!',
 'El numero de serie del motor es correcto!',
 'La linea del vehiculo es distinta. Favor de verificar',
 'El numero de motor del vehiculo es distinto. Favor de verificar',
 'La Credencial para Votar es vigente']

## Caso 3

In [32]:
chain.invoke(
    input={
        "factura": './data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 3_TK 63140-1 FAC_FRENTE_otsu.jpg',
        "tc": './data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 3_TK 63140-5 TC_FRENTE_crop_binary.jpg',
        "ine": './data/bronze/BASE_AUTOAVANZA/documentos_clean/INE_FRENTE/Caso 3_TK 63140-3 INE_FRENTE_crop_binary.jpg',
    }
)

['El nombre del propietario es correcto!',
 'El NIV del vehiculo es correcto!',
 'El numero de serie del motor es correcto!',
 'La marca del vehiculo es distinto. Favor de verificar',
 'La linea del vehiculo es distinta. Favor de verificar',
 'El numero de motor del vehiculo es distinto. Favor de verificar',
 'La Credencial para Votar es vigente']

## Caso 4

In [33]:
chain.invoke(
    input={
        "factura": './data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 4_TK 63274-1 FAC_FRENTE_otsu.jpg',
        "tc": './data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 4_TK 63274-5 TC_FRENTE_crop_binary.jpg',
        "ine": './data/bronze/BASE_AUTOAVANZA/documentos_clean/INE_FRENTE/Caso 4_TK 63274-3 INE_FRENTE_crop_binary.jpg',
    }
)

['El nombre del propietario es correcto!',
 'El NIV del vehiculo es correcto!',
 'El numero de serie del motor es correcto!',
 'La linea del vehiculo es distinta. Favor de verificar',
 'El numero de motor es correcto!',
 'La Credencial para Votar es vigente']

## Caso 5

In [34]:
chain.invoke(
    input={
        "factura": './data/bronze/BASE_AUTOAVANZA/documentos_clean/FAC_FRENTE/Caso 5_TK 63908-2 FAC_FRENTE_otsu.jpg',
        "tc": './data/bronze/BASE_AUTOAVANZA/documentos_clean/INE_FRENTE/Caso 5_TK 63908-3 INE_FRENTE_crop_binary.jpg',
        "ine": './data/bronze/BASE_AUTOAVANZA/documentos_clean/TC_FRENTE/Caso 5_TK 63908-5 TC_FRENTE_crop_binary.jpg',
    }
)

['El nombre no aparece correctamente en los documentos',
 'El NIV es distinto. Se tiene que rechazar el tramite!',
 'El numero de serie del motor no coincide',
 'La marca del vehiculo es distinto. Favor de verificar',
 'La linea del vehiculo es distinta. Favor de verificar',
 'El numero de motor del vehiculo es distinto. Favor de verificar',
 'La Credencial para Votar no esta vigente']